In [1]:
# Importamos librerías externas: 

# Para arrancar el proyecto, necesitamos nuestras herramientas principales, que son las librerías que amplían las capacidades de Python.
# Las librerías son herramientas creadas por la comunidad que amplían Python y se instalan con pip.

# Conectar y manejar Bases de Datos MySQL. (Necesario para Fases 2 y 3)
import mysql.connector

# El motor matemático: maneja números y cálculos a gran velocidad. (Útil, aunque no esencial para este ejercicio porque no haremos cálculos complejos)
import numpy as np

# El Excel de Python: organiza y analiza datos en forma de tablas (DataFrames). (Esencial para guardar las películas)
import pandas as pd

# para poder visualizar todas las columnas de los DataFrames (Muy útil cuando trabajas con DataFrames de Pandas que tienen muchas columnas)
pd.set_option('display.max_columns', None) 

# El cartero de Internet: pide y envía información a páginas web y servidores. (Esencial para descargar la API)
import requests

# Manual de "Qué hacer si falla": identifica problemas al conectar a MySQL. (Buena práctica para manejar errores)
from mysql.connector import Error

In [2]:
# FASE 1: EXTRACCIÓN DE DATOS DE PELICULAS DESDE LA API

# Guardamos la variable 'url' la dirección web donde se encuentran los datos (la API).
url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

print(f"Intentando conectar a la API en: {url}")

# Utilizamos un bloque try/except para manejar posibles fallos de conexión a Internet.
try:
    # Hacemos una petición GET a la URL usando la librería 'requests' para descargar los datos.
    datos = requests.get(url)

    # Consultamos el código de respuesta del servidor (200 = OK).
    if datos.status_code == 200:
        print("Conexión exitosa. Código de estado:", datos.status_code)

        # Vemos el contenido de la respuesta codificada en bytes.
        datos.content
        
        # Con JSON, primero, se descodifica la secuencia de bytes y luego se estructuran los datos ( lista o diccionario)
        datos_pelis = datos.json()

        # Visualizamos la estructura de los datos descargados.
        print(f"Estructura de los datos descargados en formato json:{datos_pelis}")

    else:
        # Si el código de estado no es 200, mostramos un mensaje de error.
        print(f"Error en la conexión. Código de estado: {datos.status_code}. No se pudieron extraer los datos.")

# Cuando se produzcan estas situaciones: 1) No tener conexión a Internet (el error más común). 2) Escribir una URL mal formada (por ejemplo, faltan caracteres). 3) Que el servidor de la API esté caído o no responda a tiempo.
# Se encuentra dentro de la documentación oficial de la librería requests de Python
except requests.exceptions.RequestException as e:
    print(f"Ocurrió un error al intentar conectar a la API: {e}")

Intentando conectar a la API en: https://beta.adalab.es/resources/apis/pelis/pelis.json
Conexión exitosa. Código de estado: 200
Estructura de los datos descargados en formato json:[{'id': 1, 'titulo': 'The Godfather', 'año': 1972, 'duracion': 175, 'genero': 'Crimen', 'adultos': False, 'subtitulos': ['es', 'en']}, {'id': 2, 'titulo': 'The Godfather Part II', 'año': 1974, 'duracion': 202, 'genero': 'Crimen', 'adultos': False, 'subtitulos': ['es', 'en']}, {'id': 3, 'titulo': 'Pulp Fiction', 'año': 1994, 'duracion': 154, 'genero': 'Crimen', 'adultos': True, 'subtitulos': ['es', 'en']}, {'id': 4, 'titulo': 'Forrest Gump', 'año': 1994, 'duracion': 142, 'genero': 'Drama', 'adultos': False, 'subtitulos': ['es', 'en', 'fr']}, {'id': 5, 'titulo': 'The Dark Knight', 'año': 2008, 'duracion': 152, 'genero': 'Acción', 'adultos': False, 'subtitulos': ['es', 'en']}, {'id': 6, 'titulo': 'Fight Club', 'año': 1999, 'duracion': 139, 'genero': 'Drama', 'adultos': True, 'subtitulos': ['es', 'en']}, {'id': 7

In [3]:
# Aunque sabemos que el tipo de dato es una lista porque lo hemos visto al imprimir datos_pelis, generalmente no es recomendable eliminar la verificación del tipo de dato.
print(f"El tipo de dato es: {type(datos_pelis)}")

El tipo de dato es: <class 'list'>


In [4]:
# Usamos pandas para crear el DataFrame (el "Excel de Python") a partir de la lista de películas.
df_peliculas = pd.DataFrame(datos_pelis)

# Mostramos las filas del Dataframe para verificar que se han cargado correctamente.
print(f"Filas del Dataframe de películas:{df_peliculas}") 

# Muestra el número de películas extraídas (deberían ser 100).
print(f"El número de películas extraídas es: {len(df_peliculas)}")

Filas del Dataframe de películas:     id                                 titulo   año  duracion    genero  \
0     1                          The Godfather  1972       175    Crimen   
1     2                  The Godfather Part II  1974       202    Crimen   
2     3                           Pulp Fiction  1994       154    Crimen   
3     4                           Forrest Gump  1994       142     Drama   
4     5                        The Dark Knight  2008       152    Acción   
..  ...                                    ...   ...       ...       ...   
95   96                        La vita è bella  1997       116     Drama   
96   97                    Requiem for a Dream  2000       102     Drama   
97   98                                Memento  2000       113  Thriller   
98   99  Eternal Sunshine of the Spotless Mind  2004       108     Drama   
99  100                           Donnie Darko  2001       113     Drama   

    adultos    subtitulos  
0     False      [es, en] 

In [ ]:
# Cuando una columna tiene listas (['es', 'en']), no se pueden buscar sus elementos directamente.
# Convertir la lista a texto con astype(str) permite luego hacer búsquedas tipo LIKE
df_peliculas['subtitulos'] = df_peliculas['subtitulos'].astype(str)
df_peliculas['subtitulos']

0           ['es', 'en']
1           ['es', 'en']
2           ['es', 'en']
3     ['es', 'en', 'fr']
4           ['es', 'en']
             ...        
95    ['es', 'en', 'it']
96          ['es', 'en']
97          ['es', 'en']
98          ['es', 'en']
99          ['es', 'en']
Name: subtitulos, Length: 100, dtype: object

In [ ]:
# Verificamos la versión de la librería mysql-connector-python instalada.
print(f"Versión: {mysql.connector.__version__}")

In [ ]:
# FASE 2: CREACIÓN DE LA BASE DE DATOS (VERSIÓN PYTHON)
# A. Conexión a MySQL
try:
    # Intentamos conectarnos al servidor MySQL
    cnx = mysql.connector.connect(
        host='127.0.0.1',    # Dirección del servidor MySQL (localhost)
        user='root',         # Usuario con el que nos conectamos
        password='AdalabAlumna',  # Contraseña del usuario 
        use_pure=True, # Importante para Python 3.12 ( necesario para no romperse el Kernel)
    )
    print('Conexión exitosa')  # Si no hay errores, mostramos que la conexión fue correcta
except Error as e:  # Captura cualquier error relacionado con MySQL
    print('Error al conectar:', e)  # Muestra el mensaje de error para entender qué falló

In [ ]:
# B. Creación de la base de datos

try:
    # Creamos un cursor (intermediario entre Python y MySQL)
    mycursor = cnx.cursor()

    # Consulta SQL para crear la base de datos si no existe
    query_create_database = "CREATE DATABASE IF NOT EXISTS peliculas_db"

    # Ejecutamos la consulta SQL
    mycursor.execute(query_create_database)
    print("Base de datos creada correctamente (o ya existía).")

except mysql.connector.Error as e:
    print("Ocurrió un error al crear la base de datos:", e)

In [ ]:
# C. Creación de las tablas

try:
    # Seleccionamos la base de datos donde vamos a trabajar
    mycursor.execute("USE peliculas_db")

    # Comando SQL para crear la tabla 'peliculas' si no existe
    query_creacion_tabla = """
    CREATE TABLE IF NOT EXISTS peliculas (
        id INT AUTO_INCREMENT PRIMARY KEY,   # Identificador único
        titulo VARCHAR(255) NOT NULL,        # Título obligatorio
        año YEAR,                            # Año de estreno
        duracion INT,                        # Duración en minutos
        genero VARCHAR(100),                 # Género de la película
        adultos TINYINT(1),                  # 0 = No adultos, 1 = Adultos
        subtitulos VARCHAR(255)              # Idioma de subtítulos
    );
    """

    # Ejecutamos la consulta SQL para crear la tabla
    mycursor.execute(query_creacion_tabla)
    print("Tabla peliculas creada correctamente (o ya existía).")

except mysql.connector.Error as e:
    # Capturamos y mostramos cualquier error de MySQL
    print("Ocurrió un error al crear la tabla:", e)


In [ ]:
# FASE 3: INSERCIÓN DE DATOS

# Query con %s como placeholders para cada columna
query_insercion = """
INSERT INTO peliculas 
(id, titulo, año, duracion, genero, adultos, subtitulos) 
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

# Cambiar valores NaN por None para que MySQL los acepte como NULL
df_limpieza = df_peliculas.replace({np.nan: None})

# Pasar el DataFrame a lista de listas
datos_peliculas = df_limpieza.values.tolist()

try:
    # Insertar todos los registros
    mycursor.executemany(query_insercion, datos_peliculas)

    # Guardar cambios
    cnx.commit()

    print("Se han insertado", mycursor.rowcount, "registros")

except Exception as e:
    print("Ocurrió un error al insertar los datos:", e)

In [ ]:
# Fase 4: OBTENER INFORMACIÓN A PARTIR DE LOS DATOS. CONSULTAS SQL.

# 1. ¿Cuántas películas tienen una duración superior a 120 minutos?
query_1 = "SELECT COUNT(id) FROM peliculas WHERE duracion > 120;"

# 2. ¿Cuántas películas incluyen subtítulos en español? 
# El único caso especial de las 9 consultas porque subtítulos, originalmente, era una lista.
# Funciona si la columna subtitulos es texto, ['es', 'en'] convertido a string (astype(str)).
# LIKE '%es%' busca 'es' en cualquier parte de la cadena.
# No se puede usar = porque "['es', 'en']" no es igual a 'es'.
query_2 = "SELECT COUNT(id) FROM peliculas WHERE subtitulos LIKE '%es%';"

# 3. ¿Cuántas películas tienen contenido adulto?
query_3 = "SELECT COUNT(id) FROM peliculas WHERE adultos = 1;"

# 4. ¿Cuál es la película más antigua registrada en la base de datos?
query_4 = "SELECT titulo, año FROM peliculas ORDER BY año ASC LIMIT 1;"

# 5. Muestra el promedio de duración de las películas agrupado por género
query_5 = "SELECT genero, AVG(duracion) FROM peliculas GROUP BY genero;"

# 6. ¿Cuántas películas por año se han registrado en la base de datos? Ordena de menor a mayor
query_6 = "SELECT año, COUNT(id) FROM peliculas GROUP BY año ORDER BY año ASC;"

# 7. ¿Cuál es el año con más películas en la base de datos?
query_7 = "SELECT año, COUNT(id) FROM peliculas GROUP BY año ORDER BY COUNT(id) DESC LIMIT 1;"

# 8. Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno
query_8 = "SELECT genero, COUNT(id) FROM peliculas GROUP BY genero;"

# 9. Muestra todas las películas cuyo título contenga la palabra "Godfather"
query_9 = "SELECT titulo FROM peliculas WHERE titulo LIKE '%Godfather%';"